In [ ]:
!pip install yfinance
!pip install pandas_datareader
!pip install matplotlib
!pip install pandas-datareader
!pip install tensorflow
!pip install Keras
!pip install scikit-learn
!pip install yfinance pyfolio

In [ ]:
#!pip install numpy==1.16.1 pandas==0.23.3 matplotlib==2.2.3 pandas-datareader==0.7.0 tensorflow==1.13.1 Keras==2.2.4 scikit-learn==0.20.3

In [ ]:
import pandas as pd
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import tensorflow
import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense,Dropout,BatchNormalization,Conv1D,Flatten,MaxPooling1D,LSTM
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

import pyfolio as pf

In [ ]:
start_date=datetime.datetime(2000, 8, 30)
end_date=datetime.datetime(2022, 8,31)

In [ ]:
yfSymbol = "GC=F"
#Import stock data
!pip install yfinance

import yfinance as yf

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
df = pdr.get_data_yahoo(yfSymbol, start=start_date, end=end_date)
df.drop("Adj Close",axis=1,inplace=True)
print(df.tail())

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(df['Close'], label="Close")
#plt.plot(y_pred_train_mix, label="prediction by mix model")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Dzienna cena zamknięcia indeksu złota GC=F " + yfSymbol + ", " + "z okresu od 2000-08-30 do 2022-08-30",fontsize=18)

plt.show()

In [ ]:
start_year=start_date.year
start_month=start_date.month
end_year=end_date.year
end_month=end_date.month

first_days=[]
# First year
for month in range(start_month,13):
    first_days.append(min(df[str(start_year)+"-"+str(month)].index))
# Other years
for year in range(start_year+1,end_year):
    for month in range(1,13):
        first_days.append(min(df[str(year)+"-"+str(month)].index))
# Last year
for month in range(1,end_month+1):
    first_days.append(min(df[str(end_year)+"-"+str(month)].index))

In [ ]:
def monthly_df(df):

    dfm=df.resample("M").mean()
    dfm=dfm[:-1] # As we said, we do not consider the month of end_date
    
    dfm["fd_cm"]=first_days[:-1]
    dfm["fd_nm"]=first_days[1:]
    dfm["fd_cm_open"]=np.array(df.loc[first_days[:-1],"Open"])
    dfm["fd_nm_open"]=np.array(df.loc[first_days[1:],"Open"])
    #dfm["fd_cm_open1"]=np.array(df.loc[first_days[:-1],"Close"])
    #dfm["fd_nm_open1"]=np.array(df.loc[first_days[1:],"Close"])
    dfm["quot"]=dfm["fd_nm_open"].divide(dfm["fd_cm_open"])
    #dfm["quot1"]=dfm["fd_nm_open1"].divide(dfm["fd_cm_open1"])
    
    dfm["mv_avg_12"]= dfm["Open"].rolling(window=12).mean().shift(1)
    dfm["mv_avg_24"]= dfm["Open"].rolling(window=24).mean().shift(1)
    
    dfm=dfm.iloc[24:,:] # we remove the first 24 months, since they do not have the 2-year moving average
    
    return dfm

In [ ]:
dfm=monthly_df(df)

print(dfm.head())
print(dfm.tail())

#each month of dfm contains the moving averages of the previous 12 and 24 months (excluding the current month)
print(dfm.loc["2015-03","mv_avg_12"])
print(dfm.loc["2010-03":"2011-02","Open"])
print(dfm.loc["2010-03":"2011-02","Open"].mean())

In [ ]:
def yield_gross(df,v):
    prod=(v*df["quot"]+1-v).prod()
    n_years=len(v)/12
    return (prod-1)*100,((prod**(1/n_years))-1)*100

In [ ]:
#def cumulative_returns(df,v):
  #(returns + 1.0).cumprod()
    #prod=(v*df["quot"]+1-v).prod()
    #n_years=len(v)/12
    #return (prod-1)*100,((prod**(1/n_years))-1)*100

In [1]:
#tax_cg=0.23
tax_cg=0
#comm_bk=0.001
comm_bk=0

In [ ]:
def separate_ones(u):
    
    u_ = np.r_[0,u,0]
    i = np.flatnonzero(u_[:-1] != u_[1:])
    v,w = i[::2],i[1::2]
    if len(v)==0:
        return np.zeros(len(u)),0
    
    n,m = len(v),len(u)
    o = np.zeros(n*m,dtype=int)

    r = np.arange(n)*m
    o[v+r] = 1

    if w[-1] == m:
        o[w[:-1]+r[:-1]] = -1
    else:
        o[w+r] -= 1

    out = o.cumsum().reshape(n,-1)
    return out,n

In [ ]:
u=np.array([0,1,1,0,1,1,1,0,1])

In [ ]:
separate_ones(u)

In [ ]:
u=np.array([1,1,1,1,1,1,1,1,1])

In [ ]:
separate_ones(u)

In [ ]:
u=np.array([1,1,0,0,1,1,1,1])

In [ ]:
separate_ones(u)

In [ ]:
def yield_net(df,v):
    n_years=len(v)/12
    
    w,n=separate_ones(v)
    A=(w*np.array(df["quot"])+(1-w)).prod(axis=1)  # A is the product of each group of ones of 1 for df["quot"]
    A1p=np.maximum(0,np.sign(A-1)) # vector of ones where the corresponding element if  A  is > 1, other are 0
    Ap=A*A1p # vector of elements of A > 1, other are 0
    Am=A-Ap # vector of elements of A <= 1, other are 0
    An=Am+(Ap-A1p)*(1-tax_cg)+A1p
    prod=An.prod()*((1-comm_bk)**(2*n)) 
    
    return (prod-1)*100,((prod**(1/n_years))-1)*100   

In [ ]:
def create_window(data, window_size = 1):    
    data_s = data.copy()
    for i in range(window_size):
        data = pd.concat([data, data_s.shift(-(i + 1))], axis = 1)
        
    data.dropna(axis=0, inplace=True)
    return(data)

In [ ]:
df_dates = dfm.index
print(df_dates)

In [ ]:
def data_to_model(dfm):
    scaler=MinMaxScaler(feature_range=(0,1))
    dg=pd.DataFrame(scaler.fit_transform(dfm[["High","Low","Open","Close","Volume","fd_cm_open",\
                                          "mv_avg_12","mv_avg_24","fd_nm_open"]].values))
    print(dg)
    X=dg[[0,1,2,3,4,5,6,7]]
    print(X)
    X=create_window(X,window)
    X=np.reshape(X.values,(X.shape[0],window+1,8))
    
    y=np.array(dg[8][window:])
    
    return X,y

In [ ]:
window=5

X,y=data_to_model(dfm)
print(X.shape,y.shape)

In [ ]:
print(len(X_train))

In [ ]:
mtest=72

X_train=X[:-mtest-1,:,:]
X_test=X[-mtest-1:,:,:]
y_train=y[:-mtest-1]
y_test=y[-mtest-1:]

In [ ]:
def model_lstm1(window,features):
    
    model=Sequential()
    model.add(LSTM(400, input_shape = (window,features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(250,  return_sequences=False)) # there is no need to specify input_shape here
    model.add(Dropout(0.5))
    model.add(Dense(125,kernel_initializer='uniform',activation='relu'))        
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    
    return model

In [ ]:
model_lstm1=model_lstm1(window+1,8)
print(model_lstm1.summary())

In [ ]:
def model_lstm2(window,features):
    
    model=Sequential()
    model.add(LSTM(200, input_shape = (window,features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(100,  return_sequences=False)) # there is no need to specify input_shape here
    model.add(Dropout(0.5))
    model.add(Dense(50,kernel_initializer='uniform',activation='relu'))        
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    
    return model

In [ ]:
model_lstm2=model_lstm2(window+1,8)
print(model_lstm2.summary())

In [ ]:
def model_lstm3(window,features):
    
    model=Sequential()
    model.add(LSTM(200, input_shape = (window,features), return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(50,kernel_initializer='uniform',activation='relu'))        
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    
    return model

In [ ]:
model_lstm3=model_lstm3(window+1,8)
print(model_lstm3.summary())

In [ ]:
def model_lstm4(window,features):
    
    model=Sequential()
    model.add(LSTM(200, input_shape = (window,features), return_sequences=False))     
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    
    return model

In [ ]:
model_lstm4=model_lstm4(window+1,8)
print(model_lstm4.summary())

In [ ]:
#def model_mix(window,features):
    
#    model=Sequential()
#    model.add(Conv1D(input_shape=(window,features),filters=32,kernel_size=2,strides=1,activation='relu',padding='same'))
#    model.add(Conv1D(filters=64,kernel_size=2,strides=1,activation='relu',padding='same'))
#    model.add(LSTM(300, return_sequences=True))
#    model.add(Dropout(0.5))
#    model.add(LSTM(200,  return_sequences=False))
#    model.add(Dropout(0.5))
#    model.add(Dense(100,kernel_initializer='uniform',activation='relu'))        
#    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
#    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    
#    return model

In [ ]:
def model_lstm5(window,features):
    
    model=Sequential()
    model.add(LSTM(100, input_shape = (window,features), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100,  return_sequences=True)) # there is no need to specify input_shape here
    model.add(Dropout(0.2))
    model.add(LSTM(100,  return_sequences=True)) # there is no need to specify input_shape here
    model.add(Dropout(0.2))
    model.add(LSTM(100,  return_sequences=False)) # there is no need to specify input_shape here
    model.add(Dropout(0.2))
    model.add(Dense(20,kernel_initializer='uniform',activation='relu'))        
    model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
    model.compile(loss='mse',optimizer=Adam(lr=0.001))
    
    return model



In [ ]:
model_lstm5=model_lstm5(window+1,8)
print(model_lstm5.summary())

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=25, verbose=1,\
                                                 factor=0.25, min_lr=0.00001)

In [ ]:
history_lstm1=model_lstm1.fit(X_train,y_train,epochs=400, batch_size=24, validation_data=(X_test, y_test), \
                  verbose=1, callbacks=[learning_rate_reduction],shuffle=False)

In [ ]:
history_lstm2=model_lstm2.fit(X_train,y_train,epochs=400, batch_size=24, validation_data=(X_test, y_test), \
                  verbose=1, callbacks=[learning_rate_reduction],shuffle=False)

In [ ]:
history_lstm3=model_lstm3.fit(X_train,y_train,epochs=400, batch_size=24, validation_data=(X_test, y_test), \
                  verbose=1, callbacks=[learning_rate_reduction],shuffle=False)

In [ ]:
history_lstm4=model_lstm4.fit(X_train,y_train,epochs=400, batch_size=24, validation_data=(X_test, y_test), \
                  verbose=1, callbacks=[learning_rate_reduction],shuffle=False)

In [ ]:
history_lstm5=model_lstm5.fit(X_train,y_train,epochs=400, batch_size=24, validation_data=(X_test, y_test), \
                  verbose=1, callbacks=[learning_rate_reduction],shuffle=False)

In [ ]:
plt.plot(history_lstm1.history['loss'])
plt.plot(history_lstm1.history['val_loss'])
plt.title('model_1')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history_lstm2.history['loss'])
plt.plot(history_lstm2.history['val_loss'])
plt.title('model_2')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history_lstm3.history['loss'])
plt.plot(history_lstm3.history['val_loss'])
plt.title('model_3')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history_lstm4.history['loss'])
plt.plot(history_lstm4.history['val_loss'])
plt.title('model_4')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history_lstm5.history['loss'])
plt.plot(history_lstm5.history['val_loss'])
plt.title('model_5')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
#model_lstm.save_weights("lstm_weights.h5")

In [ ]:
#model_mix=model_mix(window+1,8)
#print(model_mix.summary())

In [ ]:
#history_mix=model_mix.fit(X_train,y_train,epochs=400, batch_size=24, validation_data=(X_test, y_test), \
                 # verbose=1, callbacks=[learning_rate_reduction],shuffle=False)

In [ ]:
#plt.plot(history_mix.history['loss'])
#plt.plot(history_mix.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper right')
#plt.show()

In [ ]:
#model_mix.save_weights("mix_weights.h5")

In [ ]:

y_pred_train_lstm1=model_lstm1.predict(X_train)
y_pred_train_lstm2=model_lstm2.predict(X_train)
y_pred_train_lstm3=model_lstm3.predict(X_train)
y_pred_train_lstm4=model_lstm4.predict(X_train)
y_pred_train_lstm5=model_lstm5.predict(X_train)
#y_pred_train_mix=model_mix.predict(X_train)

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_train, y_pred_train_lstm1)
print("mean_absolute_error")
print(MAE)

from sklearn.metrics import mean_absolute_percentage_error
MAPE = mean_absolute_percentage_error(y_train, y_pred_train_lstm1)
print("mean_absolute_percentage_error")
print(MAPE)

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_train, y_pred_train_lstm2)
print("mean_absolute_error")
print(MAE)

from sklearn.metrics import mean_absolute_percentage_error
MAPE = mean_absolute_percentage_error(y_train, y_pred_train_lstm2)
print("mean_absolute_percentage_error")
print(MAPE)

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_train, y_pred_train_lstm3)
print("mean_absolute_error")
print(MAE)

from sklearn.metrics import mean_absolute_percentage_error
MAPE = mean_absolute_percentage_error(y_train, y_pred_train_lstm3)
print("mean_absolute_percentage_error")
print(MAPE)

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_train, y_pred_train_lstm4)
print("mean_absolute_error")
print(MAE)

from sklearn.metrics import mean_absolute_percentage_error
MAPE = mean_absolute_percentage_error(y_train, y_pred_train_lstm4)
print("mean_absolute_percentage_error")
print(MAPE)

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_train, y_pred_train_lstm5)
print("mean_absolute_error")
print(MAE)

from sklearn.metrics import mean_absolute_percentage_error
MAPE = mean_absolute_percentage_error(y_train, y_pred_train_lstm5)
print("mean_absolute_percentage_error")
print(MAPE)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(y_train, label="Cena otwarcia")
plt.plot(y_pred_train_lstm1, label="Cena obiczona przez model_1")
plt.plot(y_pred_train_lstm2, label="Cena obiczona przez model_2")
plt.plot(y_pred_train_lstm3, label="Cena obiczona przez model_3")
plt.plot(y_pred_train_lstm4, label="Cena obiczona przez model_4")
plt.plot(y_pred_train_lstm5, label="Cena obiczona przez model_5")
#plt.plot(y_pred_train_mix, label="prediction by mix model")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Wynik modeli na zbiorze walidacyjnym",fontsize=25)
plt.show()

In [ ]:
#All_y_pred_train_lstm = [y_pred_train_lstm,y_pred_train_lstm1,y_pred_train_lstm2,y_pred_train_lstm3]
#print(All_y_pred_train_lstm)

print(y_pred_train_lstm1.size)
print(y_pred_train_lstm2.size)
print(y_pred_train_lstm3.size)
print(y_pred_train_lstm4.size)
print(y_pred_train_lstm5.size)

In [ ]:

y_pred_lstm1=model_lstm1.predict(X_test)
y_pred_lstm2=model_lstm2.predict(X_test)
y_pred_lstm3=model_lstm3.predict(X_test)
y_pred_lstm4=model_lstm4.predict(X_test)
y_pred_lstm5=model_lstm5.predict(X_test)
#y_pred_mix=model_mix.predict(X_test)

In [ ]:


w_lstm1=np.diff(y_pred_lstm1.reshape(y_pred_lstm1.shape[0]),1)
v_lstm1=np.maximum(np.sign(w_lstm1),0)
print(v_lstm1)

w_lstm2=np.diff(y_pred_lstm2.reshape(y_pred_lstm2.shape[0]),1)
v_lstm2=np.maximum(np.sign(w_lstm2),0)
print(v_lstm2)

w_lstm3=np.diff(y_pred_lstm3.reshape(y_pred_lstm3.shape[0]),1)
v_lstm3=np.maximum(np.sign(w_lstm3),0)
print(v_lstm3)

w_lstm4=np.diff(y_pred_lstm4.reshape(y_pred_lstm4.shape[0]),1)
#print(w_lstm)
v_lstm4=np.maximum(np.sign(w_lstm4),0)
print(v_lstm4)

w_lstm5=np.diff(y_pred_lstm5.reshape(y_pred_lstm5.shape[0]),1)
#print(w_lstm)
v_lstm5=np.maximum(np.sign(w_lstm5),0)
print(v_lstm5)

#w_mix=np.diff(y_pred_mix.reshape(y_pred_mix.shape[0]),1)
#v_mix=np.maximum(np.sign(w_mix),0)

In [ ]:
plt.figure(figsize=(60,20))
plt.plot(y_test, label="Cena otwarcia")



plt.plot(y_pred_lstm1, label="Cena obiczona przez model_1")
plt.plot(v_lstm1,label="Otwarcie i zamknięcie pozycji model_1")

plt.plot(y_pred_lstm2, label="Cena obiczona przez model_2")
plt.plot(v_lstm2,label="Otwarcie i zamknięcie pozycji model_2")

plt.plot(y_pred_lstm3, label="Cena obiczona przez model_3")
plt.plot(v_lstm3,label="Otwarcie i zamknięcie pozycji model_3")

plt.plot(y_pred_lstm4, label="Cena obiczona przez model_4")
plt.plot(v_lstm4,label="Otwarcie i zamknięcie pozycji model_4")

plt.plot(y_pred_lstm5, label="Cena obiczona przez model_5")
plt.plot(v_lstm5,label="Otwarcie i zamknięcie pozycji model_5")

#plt.plot(y_pred_mix, label="prediction mix")
#plt.plot(v_mix,label="In and out mix")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Cena otwarcia oraz otwarcie i zamknięcie pozycji przez modele",fontsize=25)
plt.show()

In [ ]:
test=dfm.iloc[-mtest:,:] 
v_bh=np.ones(test.shape[0])
v_ma=test["fd_cm_open"]>test["mv_avg_12"]

In [ ]:
def gross_portfolio(df,w):
    portfolio=[ (w*df["quot"]+(1-w))[:i].prod() for i in range(len(w))]
    return portfolio

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(gross_portfolio(test,v_bh),label="Portfolio 'Buy and Hold'")
plt.plot(gross_portfolio(test,v_ma),label="Portfolio Średniej ruchomej")

plt.plot(gross_portfolio(test,v_lstm1),label="Portfolio model_1")
plt.plot(gross_portfolio(test,v_lstm2),label="Portfolio model_2")
plt.plot(gross_portfolio(test,v_lstm3),label="Portfolio model_3")
plt.plot(gross_portfolio(test,v_lstm4),label="Portfolio model_4")
plt.plot(gross_portfolio(test,v_lstm5),label="Portfolio model_5")
#plt.plot(gross_portfolio(test,v_mix),label="Portfolio Mix")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("Gross portfolios of three methods", fontsize=25)
plt.show()

In [ ]:
print("Wyniki {:.2f} letniego testu z okresu od {} do {} \n".format(len(v_bh)/12,str(test.loc[test.index[0],"fd_cm"])[:10],\
str(test.loc[test.index[-1],"fd_nm"])[:10]))

results=pd.DataFrame({})
#results["Method"]=["Buy and hold","Moving average","LSTM","Mix"]
#results["Method"]=["Buy and hold","Moving average","LSTM"]
results["Metoda"]=["'Buy and hold'","Średnia ruchoma","model_1","model_2","model_3","model_4","model_5"]

#vs=[v_bh,v_ma,v_lstm,v_mix]
vs=[v_bh,v_ma,v_lstm1,v_lstm2,v_lstm3,v_lstm4,v_lstm5]
results["Całkowity zysk brutto"]=[str(round(yield_gross(test,vi)[0],2))+" %" for vi in vs]
results["Roczny zysk brutto"]=[str(round(yield_gross(test,vi)[1],2))+" %" for vi in vs]
#results["Total net yield"]=[str(round(yield_net(test,vi)[0],2))+" %" for vi in vs]
#results["Annual net yield"]=[str(round(yield_net(test,vi)[1],2))+" %" for vi in vs]

print(results)

In [ ]:
startSimulation = print(str(test.loc[test.index[0],"fd_cm"])[:10])
endSimulation = print(str(test.loc[test.index[-1],"fd_nm"])[:10])

In [ ]:
datesSimulation = test.index
print(type(datesSimulation))
#print(df_dates.loc["2010-03":"2011-02"])

In [ ]:
#returns
print(test['quot'])
print(len(test['quot']))

In [ ]:
data1 = test['quot'].shift(-1) * v_lstm

# Drop the missing values
data1.dropna(inplace=True)
data1

In [ ]:
#signal
print(v_lstm)
print(len(v_lstm))

In [ ]:
print(len(datesSimulation))
print(len(y_pred_train_lstm))
#print(y_pred_train_lstm)
#df1 = pd.DataFrame(y_pred_train_lstm)
#print(type(df1))
#df1 = df1.squeeze()
#datesSimulation['outOfSimulation'] = y_pred_train_lstm
#df1 = y_pred_train_lstm.pct_change()
#df['target'] = np.where(df['returns'].shift(-1) > 0, 1, 0)
#print(datesSimulation.head())
#print(datesSimulation.tail())

In [ ]:
df1_returns = df1.pct_change()
df1_returns.dropna(inplace=True)
print(df1_returns)
print(type(df1_returns))

In [ ]:
def cumulative_returns(returns):

    res = (returns + 1.0).cumprod()
    res.columns = ['cumulative return']

    return res
cret = cumulative_returns(df1_returns)
print(cret)


In [ ]:
plt.figure(figsize=(30,10))
plt.plot(cret, label="cumulative_returns")
#plt.plot(y_pred_train_mix, label="prediction by mix model")
plt.legend(fontsize=20)
plt.grid(axis="both")
plt.title("cumulative_returns v_lstm",fontsize=18)

plt.show()